In [1]:
# pip install nb_black
%load_ext lab_black
import pandas as pd
import numpy as np

In [2]:
pop_2000_2009 = pd.read_csv(
    "../00_source/CensusEstimates2000-2010.csv", encoding="latin-1"
)
pop_2010_2019 = pd.read_csv(
    "../00_source/CensusEstimates2010-2019.csv", encoding="latin-1"
)

In [3]:
pop_2000_2009.info()
# No missing values

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3194 entries, 0 to 3193
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   SUMLEV             3194 non-null   int64 
 1   REGION             3194 non-null   int64 
 2   DIVISION           3194 non-null   int64 
 3   STATE              3194 non-null   int64 
 4   COUNTY             3194 non-null   int64 
 5   STNAME             3194 non-null   object
 6   CTYNAME            3194 non-null   object
 7   ESTIMATESBASE2000  3194 non-null   int64 
 8   POPESTIMATE2000    3194 non-null   int64 
 9   POPESTIMATE2001    3194 non-null   int64 
 10  POPESTIMATE2002    3194 non-null   int64 
 11  POPESTIMATE2003    3194 non-null   int64 
 12  POPESTIMATE2004    3194 non-null   int64 
 13  POPESTIMATE2005    3194 non-null   int64 
 14  POPESTIMATE2006    3194 non-null   int64 
 15  POPESTIMATE2007    3194 non-null   int64 
 16  POPESTIMATE2008    3194 non-null   int64 


In [4]:
# 2000-2009
pop_2000_2009 = pop_2000_2009[
    [
        "STNAME",
        "CTYNAME",
        "POPESTIMATE2000",
        "POPESTIMATE2001",
        "POPESTIMATE2002",
        "POPESTIMATE2003",
        "POPESTIMATE2004",
        "POPESTIMATE2005",
        "POPESTIMATE2006",
        "POPESTIMATE2007",
        "POPESTIMATE2008",
        "POPESTIMATE2009",
    ]
]
# pop_2000_2009.head()

# 2010-2019
pop_2010_2019 = pop_2010_2019[
    [
        "STNAME",
        "CTYNAME",
        "CENSUS2010POP",
        "POPESTIMATE2011",
        "POPESTIMATE2012",
        "POPESTIMATE2013",
        "POPESTIMATE2014",
        "POPESTIMATE2015",
        "POPESTIMATE2016",
        "POPESTIMATE2017",
        "POPESTIMATE2018",
        "POPESTIMATE2019",
    ]
]

print(len(pop_2000_2009), len(pop_2010_2019))
pop_2010_2019.rename(columns={"CENSUS2010POP": "CENSUS2010"}, inplace=True)

3194 3193


In [5]:
pop_2000_2009_duplicates = pop_2000_2009[pop_2000_2009.duplicated(keep=False)]
print(f"2000-2009 Duplicates:\n\n{pop_2000_2009_duplicates}\n\n")
pop_2010_2019_duplicates = pop_2010_2019[pop_2010_2019.duplicated(keep=False)]
print(f"2010-2019 Duplicates:\n\n{pop_2010_2019_duplicates}")

2000-2009 Duplicates:

                   STNAME               CTYNAME  POPESTIMATE2000  \
327  District of Columbia  District of Columbia           572046   
328  District of Columbia  District of Columbia           572046   

     POPESTIMATE2001  POPESTIMATE2002  POPESTIMATE2003  POPESTIMATE2004  \
327           574504           573158           568502           567754   
328           574504           573158           568502           567754   

     POPESTIMATE2005  POPESTIMATE2006  POPESTIMATE2007  POPESTIMATE2008  \
327           567136           570681           574404           580236   
328           567136           570681           574404           580236   

     POPESTIMATE2009  
327           592228  
328           592228  


2010-2019 Duplicates:

                   STNAME               CTYNAME  CENSUS2010  POPESTIMATE2011  \
327  District of Columbia  District of Columbia      601723           619800   
328  District of Columbia  District of Columbia      601723       

#### Exact duplicate of DC in both...okay to drop

In [6]:
pop_2000_2009.drop(328, inplace=True)
pop_2010_2019.drop(328, inplace=True)
assert not pop_2000_2009.duplicated().any()
assert not pop_2010_2019.duplicated().any()

In [7]:
print(len(pop_2000_2009), len(pop_2010_2019))

3193 3192


In [8]:
pop_00_09 = pd.melt(pop_2000_2009, ["STNAME", "CTYNAME"])
assert not pop_00_09.duplicated().any()

pop_10_19 = pd.melt(pop_2010_2019, ["STNAME", "CTYNAME"])
assert not pop_10_19.duplicated().any()

In [9]:
pop_00_09.rename(columns={"variable": "YEAR", "value": "POP"}, inplace=True)
pop_10_19.rename(columns={"variable": "YEAR", "value": "POP"}, inplace=True)

In [10]:
pop_00_09["YEAR"] = pop_00_09["YEAR"].apply(lambda x: x[-4:])
pop_10_19["YEAR"] = pop_10_19["YEAR"].apply(lambda x: x[-4:])

In [11]:
states = ["Florida", "Texas", "Washington"]

for state in states:
    len1 = len(pop_00_09[pop_00_09["STNAME"] == state])
    len2 = len(pop_10_19[pop_10_19["STNAME"] == state])
    print(len1)
    print(len2)
    assert len1 == len2
    print(state)

680
680
Florida
2550
2550
Texas
400
400
Washington


#### Our target states have the same number of counties in both population sets

In [12]:
pop_10_19.head()

,STNAME,CTYNAME,YEAR,POP
0,Alabama,Alabama,2010,4779736
1,Alabama,Autauga County,2010,54571
2,Alabama,Baldwin County,2010,182265
3,Alabama,Barbour County,2010,27457
4,Alabama,Bibb County,2010,22915


In [13]:
merge = pd.merge(
    pop_00_09, pop_10_19, on=["STNAME", "CTYNAME"], how="outer", indicator=True
)
# assert merge[merge["_merge"] != "both"].isnull().all().all()

#### There are uneven counts in each dataset

In [14]:
disjoint = (
    merge[merge["_merge"] != "both"]
    .sort_values(by=["STNAME", "CTYNAME"])[["STNAME", "CTYNAME", "_merge"]]
    .value_counts()
)

disjoint

STNAME        CTYNAME                   _merge    
Virginia      Bedford city              left_only     10
South Dakota  Shannon County            left_only     10
              Oglala Lakota County      right_only    10
Louisiana     LaSalle Parish            right_only    10
              La Salle Parish           left_only     10
Alaska        Wade Hampton Census Area  left_only     10
              Petersburg Census Area    left_only     10
              Petersburg Borough        right_only    10
              Kusilvak Census Area      right_only    10
dtype: int64

#### Shannon County -> Oglala Lakota County (2015)
#### Need to correct La Salle Parish to LaSalle Parish
#### Wade Hampton Census Area -> Kusilvak Census Area (2010)
#### Petersburg Census Area -> Petersburg Borough
#### Bedford City incorporated into Bedford County (2013)

In [15]:
# df.loc[(df['county'] == "old name") & (df['state'] == 'tx'), 'county'] =new

In [16]:
replacements = [
    ("Virginia", ("Bedford city", "Bedford County")),
    ("South Dakota", ("Shannon County", "Oglala Lakota County")),
    ("Louisiana", ("La Salle Parish", "LaSalle Parish")),
    ("Alaska", ("Wade Hampton Census Area", "Kusilvak Census Area")),
    ("Alaska", ("Petersburg Census Area", "Petersburg Borough")),
]

In [17]:
for item in replacements:
    state, county = item
    old, new = county
    pop_00_09.loc[
        (pop_00_09["CTYNAME"] == old) & (pop_00_09["STNAME"] == state), "CTYNAME"
    ] = new

In [18]:
pop_00_09 = pop_00_09.groupby(["STNAME", "CTYNAME", "YEAR"]).sum()

In [19]:
pop_00_09.reset_index(inplace=True)

In [20]:
merge = pd.merge(
    pop_00_09, pop_10_19, on=["STNAME", "CTYNAME"], how="outer", indicator=True
)
assert merge[merge["_merge"] != "both"].isnull().all().all()

In [21]:
pop_00_19 = pop_00_09.append(pop_10_19)
pop_00_19 = pop_00_19.groupby(["STNAME", "CTYNAME", "YEAR"]).sum().reset_index()
assert not pop_00_19.duplicated().any()

In [22]:
pop_00_19.to_csv("../20_intermediate_files/FinalPopDataset.csv", index=False)